In [1]:
import pandas as pd


In [2]:
# Load the data
df = pd.read_csv(
    filepath_or_buffer="../data/trump_data.csv",
    dtype={"trump": int}
)

# Convert specific columns to lists of integers
df['hand'] = df['hand'].apply(lambda x: [int(i) for i in x.replace("[", "").replace("]", "").split(",")])
df['score'] = df['score'].apply(lambda x: [int(i) for i in x.replace("[", "").replace("]", "").split(",")])

df["score_avg"] = df["score"].apply(lambda x: sum(x) / len(x))
df["hand_id"] = df["hand"].apply(lambda hand: int(''.join(str(x) for x in hand), 2))

In [3]:
unique_hands = df["hand_id"].unique()

# train, validation, test split unique hands
split = [int(len(unique_hands) * 0.8), int(len(unique_hands) * 0.9)]

train_hands = unique_hands[:split[0]]
validation_hands = unique_hands[split[0]:split[1]]
test_hands = unique_hands[split[1]:]

train_df = df[df["hand_id"].isin(train_hands)]
validation_df = df[df["hand_id"].isin(validation_hands)]
test_df = df[df["hand_id"].isin(test_hands)]